In [ ]:
import numpy as np
import strawberryfields as sf
from strawberryfields.ops import *
import tensorflow as tf
import random as rd
import pickle

import seaborn as sns
import tensorflow as tf
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input
from datetime import datetime
from tensorflow.keras import optimizers
from scipy.special import comb
from tensorflow.keras.initializers import RandomUniform, Identity
from tensorflow.keras.layers import Dense, ReLU
from tensorflow.keras import activations
from tensorflow.keras.callbacks import LearningRateScheduler, TensorBoard
import pickle
from tensorflow.keras.optimizers.schedules import PolynomialDecay

In [ ]:
# what is the interpretation of this function and how does one get to it?
# (A): x aktuell, y1/y2 vergangenheit, memerisot change function?
def function_lag(x,y1,y2):
    return 0.4*y1+0.4*y1*y2+0.6*x**3+0.1

# what is the interpretation of this function - could one name it accordingly? Like multiply_3inputs?

# prediction fnction xt, xt-1, xt-2
def function_lag_iris(x1,x2,x3):
    return x1*x2*x3

def target_function(xt, xt1, 2):
    return 0.4*xt1+0.4*xt1*xt2+0.6*xt**3+0.1

In [ ]:
inp=np.random.random_sample(100)*0.5

In [ ]:
print("Memristor time lag")
res_mem={}
prove_num=1 #?

# Linear square und cube functions to predict
for power in [3]:
    dip=power # memory? why call it dip? 
    res_mem={}

    for p in range(prove_num):
        eng = sf.Engine(backend="tf", backend_options={"cutoff_dim": 4})
        
        # create 3 mode circuit
        circuit=sf.Program(3)

        # use random training points
        x_train=np.sqrt(inp)
        
        # translate values to angles -> encoding
        angs=2*np.arccos(x_train)
    
    
        
        print("prova %d su %d" %(p, prove_num)) # english?

        # first memoristor phase (random)
        phi1 = tf.Variable(rd.uniform(0.01,1)*2*np.pi, constraint=lambda z: tf.clip_by_value(z, 0, 2*np.pi))

        # what for? phi 2?
        x_2=tf.Variable(rd.uniform(0.01,1),  constraint=lambda z: tf.clip_by_value(z, 0.01, 1)) # free parameter in phi2, phi2 is memoristor 
        p1=tf.constant(rd.uniform(0.01,1))
        p2=tf.Variable(rd.uniform(0.01,1))

        # input training data
        phienc = tf.constant(angs)

         # second memoristor phase (random)
        phi3 = tf.Variable(rd.uniform(0.01,1)*2*np.pi, constraint=lambda z: tf.clip_by_value(z, 0, 2*np.pi))
        print("Random choices: ", phi1, phi3, x_2)
        
        phi_1, phi_2,phi_3, phi_enc = circuit.params("phi1","phi2", "phi3", "phienc")
        
        
        # for UQ stuff we want to have the circuit below with the memristor parametrized by phi_2 and then we want to make the parameters
        # for the MZIs phi_1 and phi_3 stochastic
        # that means sample phi_1_sample in np.normal(phi_1, var)  and phi_3_sample in np.normal(phi_3, var)
        # where we experiment with the var>0 and phi_1 and phi_3 are obtained from the training loop below   
            
        with circuit.context as q:
            Vac     | q[0]
            Fock(1) | q[1]
            Vac     | q[2]
        
            ###----------------------------------------------------------------------
        
            # input 
            ## encoding MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_enc)           | q[1] # encoding
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
            ## first MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_1)              | q[1] # state preparation
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
             ## second MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[1], q[2])
        
            Rgate(phi_2)              | q[1] # memoristor value
        
            BSgate(np.pi/4, np.pi/2) | (q[1], q[2])
            
            
            ## third MZI
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
        
            Rgate(phi_3)              | q[1] # state tomography
        
            BSgate(np.pi/4, np.pi/2) | (q[0], q[1])
            
            
        opt = tf.keras.optimizers.Adam(learning_rate=0.003)
        # we should also do an ablation study on the number of steps for training
        # in the error plot it looked like at least 50 steps is ok-ish
        steps = 1
        
        for step in range(steps):
        
            # reset the engine if it has already been executed
            if eng.run_progs:
                eng.reset()
        
            with tf.GradientTape() as tape:
                loss=0
                index=0
                p1=np.zeros(dip) # dip ist memory
                p2=np.zeros(dip)
                for phi in range(len(phienc)): # 0 - all datapoints
                    t=phi-index*dip # index 0 - 3 // t 0-2
                    if t==dip-1: 
                        index+=1
                    if phi==0: # if memory empty, start with sq 0.5 
                        results = eng.run(circuit, args={"phi1": phi1, "phi3": phi3, "phi2": tf.Variable(tf.acos(np.sqrt(0.5))), "phienc": phienc[phi]})
                    if phi>0: # if not empty use last values until max dip --> memoristor juhu
                        results = eng.run(circuit, args={"phi1": phi1, "phi3": phi3, "phi2": tf.acos(tf.sqrt(np.sum(p1)/dip+x_2*np.sum(p2)/dip)), "phienc": phienc[phi]})

                    # get probabilities
                    #in the UQ version we get the prob per sample of phi_1 and phi_3, meaning that for n samples we get 
                    # n prob results, but not in the training loop and initially only in the predict loop
                    prob=results.state.all_fock_probs()

                    # then we compute the mean and variance (or std) over samples
                    # so for each t we can either store two values the mean prediction and the variance (or std) or just straight
                    # up n samples and then later do the computation on these

                    # store prob in memory p1 (index at t)
                    p1[t]=tf.Variable(np.real(prob[0,1,0])) # store prob in memory p1 (index at t)
                    p2[t]=tf.Variable(np.real(prob[0,0,1]))
        
                    if phi>=2: # should be phi > dip
                        # why is the to the power 6 part left out here? because there is no memristor?
                        f2=function_lag_iris(x_train[phi]**2,x_train[phi-1]**2,x_train[phi-2]**2) # function to predict, current, past, past
                        
                        # what does the to the power 6 version do? Ask Iris?
                        #f1=x_train[phi]**6
                        #loss += (abs(f2-prob[0,1,0])+abs(f1-prob[0,0,1]))**2
                        loss += (abs(f2-prob[0,0,1]))**2

            gradients = tape.gradient(loss, [phi1, phi3, x_2]) # get gradients
            opt.apply_gradients(zip(gradients, [phi1, phi3, x_2]))
            res_mem[('loss', 'tr', step, p, power)]=[loss, phi1.value(), phi3.value(), x_2.value(), inp] # store results of memoristor for log
            print("Loss at step {}: {}".format(step+1, np.real(loss)))
        print("Final loss: %f" %np.real(loss))
        print("Optimal parameters: %f and %f\n x2= %f" %(phi1,phi3, x_2))
    file=open("results_mem_"+str(power)+"_t_lag_iris_lr_lag_try", "wb")
    pickle.dump(res_mem, file)
    file.close()

In [ ]:
#here then we can compare no memristor with sampled weights vs no memristor without sampled weights
# and the same with a memristor


# ablation study if the std or variance is useful for setting the parameter (var in the normal dist for phi_1 and phi_3)
# compute the quantiles over the std for all inputs sampled in (0,100) 
# then if in prediction mode for some x in (0,100) the std >0.8 (e.g 0.8 but let's try all quantiles)
# omit that point x from the RMSE computation

for p in ['3']:
    #file=open("function prediction/powers/results_mem_"+p+"_power_2", "rb")
    file=open("results_mem_3_t_lag_iris_lr_003_274", "rb")
    # can one write out r4m and r4nm - memristor and no memristor? reservoir 4?
    r4m=pickle.load(file)
    file.close()
    file=open("results_nomem_3_t_lag_iris_lr_003_274", "rb")
    r4nm=pickle.load(file)
    file.close()
    
    loss_memory=[]
    for x in range(10):
        loss_memory.append(r4m[('loss', 'tr',59,x, int(p))][0])
        
    index_m=np.argmin(loss_memory)
    #index_m=2
    
    loss=[]
    for x in range(10):
        loss.append(r4nm[('loss', 'tr', 59,x, int(p))][0])
        
    index_nomem=np.argmin(loss)
    r4nm[('loss', 'tr', 59,index_nomem, int(p))]
    
    r4_nm=[]
    r4_m=[]
    for i in range(60):
        r4_nm.append(r4nm[('loss', 'tr', i,index_nomem, int(p))][0])   
        r4_m.append(r4m[('loss', 'tr', i,index_m, int(p))][0])   
    sp=str(p)    
    plt.plot(np.real(np.array(r4_nm)), label=r'No memristor $f(x)=x^'+sp+'$')
    plt.plot(np.real(np.array(r4_m)), label=r'Memristor $f(x)=x^'+sp+'$')
    #plt.yticks(np.array([0,2,4,6]), fontsize=18)
    plt.xticks(np.array([0,10,20,30,40,50, 60]), fontsize=18)
    plt.xlabel("step", fontsize=23)
    plt.ylabel("loss", fontsize=23)
    plt.legend(fontsize=10)
    plt.tight_layout
print(loss_memory[index_m])
o_p=r4m['loss', 'tr',59,index_m, 3]

# is this loss "good" in any sense? Because if the interval is [0,1], anything above 10 in RMSE is shit or? or is the x range:  x in range(10)?